In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process

In [2]:
df= pd.read_excel('data/assignment.xlsx')
df.shape

(13808, 11)

In [3]:
df.head(10)

,Product_Id,Client_Itemdesc,Client_Sectionname,Brand,Size_Match,UOM_Match,Outside_brand,Outside_item_desc,long_name,Item_Description,Ingredient1
0,30087036721,pregestimil,baby formula,Enfamil,16,oz,NaN,NaN,NaN,Enfamil Pregestimil Lipil Hypoallergenic Infan...,NaN
1,2260018162,arrid xx solid cool,deodorant,XTRA,1,pk,Arrid,Arrid Xx Extra Extra Dry Cool Shower Solid Ant...,NaN,Arrid XX Cool Shower Solid Antiperspirant Deod...,NaN
2,3320019756,a&h solid umax cool,deodorant,NaN,0,oz,Arm & Hammer,Arm & Hammer Ultra Max Cool Blast Clear Gel De...,NaN,NaN,NaN
3,20019764,a&h deo umax cool blast,deodorant,Arm&Hammer,3,oz,NaN,NaN,NaN,A&H ULT MX IS CL BLST 2.6Z,NaN
4,30045089507,nzrl anti dandruff shm,shampoo/conditioners/stylers,Nizoral,7,oz,NaN,NaN,NaN,Nizoral Anti-Dandruff Shampoo,NaN
5,2260019114,arrid solid xdry mornin,deodorant,ARRID,3,oz,Arrid,Arrid Extra Dry Clear Gel Morning Clean Deodorant,NaN,"Arrid ExtraDry Antiperspirant & Deodorant, Cle...",NaN
6,89018000119,plum spnch peas pear,baby food,Plum Organics,4,oz,Plum Organics,Plum Organics Spinach Peas & Pear Organic Baby...,NaN,PLUM ORGANICS STAGE 2 SECOND BLENDS PEAR,NaN
7,-99,plum spnch peas pear,prgx,NaN,0,oz,NaN,NaN,NaN,NaN,NaN
8,89018000197,plum blubry pear,baby food,Plum Organics,4,oz,Plum Organics,"Plum Organics Organic Baby Food Blueberry, Pea...",NaN,PLUM ORGANICS STAGE 2 SECOND BLENDS PEAR,NaN
9,-99,plum blubry pear,prgx,NaN,0,oz,NaN,NaN,NaN,NaN,NaN


In [4]:
#check for NaNs
df.isna().sum()

Product_Id                0
Client_Itemdesc           0
Client_Sectionname        0
Brand                  3148
Size_Match                0
UOM_Match               187
Outside_brand          8788
Outside_item_desc      8788
long_name             13450
Item_Description       4546
Ingredient1           13451
dtype: int64

In [5]:
# Remove duplicat Product_Id's (assumption = Product_Id is unique product key)
df=df.drop_duplicates(subset = 'Product_Id', keep ='first')
df.shape

(10430, 11)

In [6]:
#Fill NaNs in Brands w/Outside_brand
df.Brand.fillna(df.Outside_brand, inplace = True)
df.isna().sum()

Product_Id                0
Client_Itemdesc           0
Client_Sectionname        0
Brand                  2634
Size_Match                0
UOM_Match               176
Outside_brand          7280
Outside_item_desc      7280
long_name             10186
Item_Description       4006
Ingredient1           10187
dtype: int64

In [7]:
#Match Brand Names w/Outside_brand
def fillVName(c):
    name_to_check = str(c.Brand)
    a = process.extract(name_to_check, df.Outside_brand.tolist(), limit=1) 
    if a[0][1]>= 80:
        return a[0][0]
    else:
        return np.nan

In [8]:
#Get matching ratio of Brand Namest to Outside_brand

def fillVmatch(c):
    name_to_check = str(c.Brand)
    a = process.extract(name_to_check, df.Outside_brand.tolist(), limit=1) 
    if a[0][1]>= 80:
        return a[0][1]
    else:
        return np.nan

In [9]:
small=df.head(1000)
%time
small['Matched_Brand']=small.apply(fillVName, axis=1)

Wall time: 0 ns


C:\Users\ae79644\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
%%time
small['Matched_Ratio']=small.apply(fillVmatch, axis=1)

Wall time: 7min 44s


C:\Users\ae79644\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
small.head(10)

,Product_Id,Client_Itemdesc,Client_Sectionname,Brand,Size_Match,UOM_Match,Outside_brand,Outside_item_desc,long_name,Item_Description,Ingredient1,Matched_Brand,Matched_Ratio
0,30087036721,pregestimil,baby formula,Enfamil,16,oz,NaN,NaN,NaN,Enfamil Pregestimil Lipil Hypoallergenic Infan...,NaN,Enfamil,100.0
1,2260018162,arrid xx solid cool,deodorant,XTRA,1,pk,Arrid,Arrid Xx Extra Extra Dry Cool Shower Solid Ant...,NaN,Arrid XX Cool Shower Solid Antiperspirant Deod...,NaN,NaN,NaN
2,3320019756,a&h solid umax cool,deodorant,Arm & Hammer,0,oz,Arm & Hammer,Arm & Hammer Ultra Max Cool Blast Clear Gel De...,NaN,NaN,NaN,Arm & Hammer,100.0
3,20019764,a&h deo umax cool blast,deodorant,Arm&Hammer,3,oz,NaN,NaN,NaN,A&H ULT MX IS CL BLST 2.6Z,NaN,Arm & Hammer,95.0
4,30045089507,nzrl anti dandruff shm,shampoo/conditioners/stylers,Nizoral,7,oz,NaN,NaN,NaN,Nizoral Anti-Dandruff Shampoo,NaN,NaN,NaN
5,2260019114,arrid solid xdry mornin,deodorant,ARRID,3,oz,Arrid,Arrid Extra Dry Clear Gel Morning Clean Deodorant,NaN,"Arrid ExtraDry Antiperspirant & Deodorant, Cle...",NaN,Arrid,100.0
6,89018000119,plum spnch peas pear,baby food,Plum Organics,4,oz,Plum Organics,Plum Organics Spinach Peas & Pear Organic Baby...,NaN,PLUM ORGANICS STAGE 2 SECOND BLENDS PEAR,NaN,Plum Organics,100.0
7,-99,plum spnch peas pear,prgx,NaN,0,oz,NaN,NaN,NaN,NaN,NaN,NaN,100.0
8,89018000197,plum blubry pear,baby food,Plum Organics,4,oz,Plum Organics,"Plum Organics Organic Baby Food Blueberry, Pea...",NaN,PLUM ORGANICS STAGE 2 SECOND BLENDS PEAR,NaN,Plum Organics,100.0
10,84667500053,plum broccoli apple,baby food,Plum Organics,4,oz,Plum Organics,Plum Organics Organic Baby Food Broccoli & App...,NaN,PLUM BABY FOOD BROCCOLI & APPLE ORGANICS,NaN,Plum Organics,100.0


In [14]:
small['Matched_Ratio'].fillna(0, inplace= True)

C:\Users\ae79644\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [16]:
small['Matched_Brand'].fillna(small.Brand, inplace= True)

In [17]:
small.head(20)

,Product_Id,Client_Itemdesc,Client_Sectionname,Brand,Size_Match,UOM_Match,Outside_brand,Outside_item_desc,long_name,Item_Description,Ingredient1,Matched_Brand,Matched_Ratio
0,30087036721,pregestimil,baby formula,Enfamil,16,oz,NaN,NaN,NaN,Enfamil Pregestimil Lipil Hypoallergenic Infan...,NaN,Enfamil,100.0
1,2260018162,arrid xx solid cool,deodorant,XTRA,1,pk,Arrid,Arrid Xx Extra Extra Dry Cool Shower Solid Ant...,NaN,Arrid XX Cool Shower Solid Antiperspirant Deod...,NaN,XTRA,0.0
2,3320019756,a&h solid umax cool,deodorant,Arm & Hammer,0,oz,Arm & Hammer,Arm & Hammer Ultra Max Cool Blast Clear Gel De...,NaN,NaN,NaN,Arm & Hammer,100.0
3,20019764,a&h deo umax cool blast,deodorant,Arm&Hammer,3,oz,NaN,NaN,NaN,A&H ULT MX IS CL BLST 2.6Z,NaN,Arm & Hammer,95.0
4,30045089507,nzrl anti dandruff shm,shampoo/conditioners/stylers,Nizoral,7,oz,NaN,NaN,NaN,Nizoral Anti-Dandruff Shampoo,NaN,Nizoral,0.0
5,2260019114,arrid solid xdry mornin,deodorant,ARRID,3,oz,Arrid,Arrid Extra Dry Clear Gel Morning Clean Deodorant,NaN,"Arrid ExtraDry Antiperspirant & Deodorant, Cle...",NaN,Arrid,100.0
6,89018000119,plum spnch peas pear,baby food,Plum Organics,4,oz,Plum Organics,Plum Organics Spinach Peas & Pear Organic Baby...,NaN,PLUM ORGANICS STAGE 2 SECOND BLENDS PEAR,NaN,Plum Organics,100.0
7,-99,plum spnch peas pear,prgx,NaN,0,oz,NaN,NaN,NaN,NaN,NaN,NaN,100.0
8,89018000197,plum blubry pear,baby food,Plum Organics,4,oz,Plum Organics,"Plum Organics Organic Baby Food Blueberry, Pea...",NaN,PLUM ORGANICS STAGE 2 SECOND BLENDS PEAR,NaN,Plum Organics,100.0
10,84667500053,plum broccoli apple,baby food,Plum Organics,4,oz,Plum Organics,Plum Organics Organic Baby Food Broccoli & App...,NaN,PLUM BABY FOOD BROCCOLI & APPLE ORGANICS,NaN,Plum Organics,100.0


In [18]:
small[small.Matched_Ratio <90]

,Product_Id,Client_Itemdesc,Client_Sectionname,Brand,Size_Match,UOM_Match,Outside_brand,Outside_item_desc,long_name,Item_Description,Ingredient1,Matched_Brand,Matched_Ratio
1,2260018162,arrid xx solid cool,deodorant,XTRA,1,pk,Arrid,Arrid Xx Extra Extra Dry Cool Shower Solid Ant...,NaN,Arrid XX Cool Shower Solid Antiperspirant Deod...,NaN,XTRA,0.0
4,30045089507,nzrl anti dandruff shm,shampoo/conditioners/stylers,Nizoral,7,oz,NaN,NaN,NaN,Nizoral Anti-Dandruff Shampoo,NaN,Nizoral,0.0
570,80011196,tc bw shea butter,bar/hand soap/ body wash,TopCare,24,oz,NaN,NaN,NaN,TC DS SHEA BUTTER BODYWSH,NaN,TopCare,0.0
571,3680006447,tc bdywsh ultra moistur,bar/hand soap/ body wash,Top Care,24,oz,NaN,NaN,NaN,Top Care Ultra Moisturizing Body Wash for Dry ...,NaN,Method Home Care,86.0
572,3680033907,tc bdywsh sensitve skin,bar/hand soap/ body wash,TopCare,24,oz,NaN,NaN,NaN,TC UNSC SENS BW(DOVE),NaN,TopCare,0.0
573,3680033910,tc bdywsh orgnal green,bar/hand soap/ body wash,TopCare,18,oz,NaN,NaN,NaN,TC BODY WASH ORIG GREEN,NaN,TopCare,0.0
580,3680011840,tc men sport bw,bar/hand soap/ body wash,TopCare,18,oz,NaN,NaN,NaN,TOPCARE PURE SPORT BW,NaN,TopCare,0.0
766,3700008834,safegrd 4bar white,bar/hand soap/ body wash,Saveguard,16,oz,NaN,NaN,NaN,SG 4BAR WHITE W/ALOE,NaN,Saveguard,0.0
933,3680077127,tc facial shts oil abso,face/acne creams/soaps,TopCare,50,pk,NaN,NaN,NaN,TC OIL ABSORB SHEETS 50CT (C&C,NaN,TopCare,0.0
937,31254778022,women rogaine foam,shampoo/conditioners/stylers,Rogaine,4,oz,NaN,NaN,NaN,Rogaine for Women Hair Regrowth Treatment Foam...,NaN,Rogaine,0.0


In [23]:
df['UOM_Match'].str.upper().value_counts()

OZ       5248
PK       2208
CT       1726
MG        268
FZ        221
CO        113
E         108
LB         95
LT         42
FLOZ       41
MM         32
ML         24
GM         23
METER      22
DAY        20
MCG        19
EA         16
YD         10
B           7
PR          4
1           2
H           2
IN          1
M           1
QT          1
Name: UOM_Match, dtype: int64

In [24]:
small['UOM_Match'].str.upper().value_counts()

OZ      889
CT       57
PK       17
E         9
FLOZ      4
FZ        3
LB        2
Name: UOM_Match, dtype: int64

In [25]:
small.to_csv('smallset.csv')

In [ ]:
def fillVUOM(c):
    name_to_check = str(c.UOM_Match)
    a = process.extract(name_to_check, df.Outside_brand.tolist(), limit=1) 
    if a[0][1]>= 80:
        return a[0][0]
    else:
        return np.nan